In [1]:
import cv2                   # to capture image
import mediapipe as mp       # to capture hand and use mutiple functions and classes related to it
import math
import numpy as np

In [2]:
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [3]:
wCam, hcam= 640, 480

to access speakers through library pycaw

In [4]:
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
#print(volume.GetMute(),"\n")
#print(volume.GetMasterVolumeLevel(),"\n")
#print(volume.GetVolumeRange())

In [5]:
mp_hands = mp.solutions.hands                        # recognize hands in the image
hands=mp_hands.Hands()                               # configuration of hand
capture = cv2.VideoCapture(0)                        #webcam input
#mp_drawing = mp.solutions.drawing_utils

In [6]:
while True:
    success, img=capture.read()                       # captured image is saved in img
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)        # captured image is converted from BGR to RGB
    results = hands.process(img)                      # hands from captured image are recongnized and processed in results
          
    if results.multi_hand_landmarks:                  # more than one hands
        for handLms in results.multi_hand_landmarks:  # for each hand run the loop and access it information
            lmList = []                               # empty lsit that will contain list of sublist of [id, cx, cy]
           # mp_drawing.draw_landmarks(img, handLms, mp_hands.HAND_CONNECTIONS) #for each hand draw all 20 landmarks
            for id, lm in enumerate(handLms.landmark):
               #print(id, lm)                          #display id=landmarks, lm=landmarks coordinates for all 20 landmarks
               h, w, c=img.shape                       # height width channel(red green blue) of captured image
               cx, cy=int(lm.x*w), int(lm.y*h)         # cx= landmark_x_coordinated * widtH_of_the_captured_image 
               #print(id, cx, cy)
               #print(id, h, w) 
               lmList.append([id, cx, cy])
               #print(lmList)
            
            if lmList:
                xpos1, ypos1= lmList[4][1],lmList[4][2] #coordinates of thumb
                xpos2, ypos2= lmList[8][1],lmList[8][2] #coordinates of forefinger
                
                #draw circles around the thumb and forefinger
                cv2.circle(img,(xpos1, ypos1), 15, (255, 0, 0), cv2.FILLED)
                cv2.circle(img,(xpos2, ypos2), 15, (255, 0, 0), cv2.FILLED)
                cv2.line(img,(xpos1, ypos1),(xpos2, ypos2), (0, 255, 0), 6)
    
                length = math.hypot(xpos2-xpos1, ypos2-ypos1) #measure distance between thumb and forefinger
                #print(length)
                
                zpos1=(xpos1 + xpos2)//2
                zpos2=(ypos1 + ypos2)//2
                cv2.circle(img,(zpos1, zpos2), 15, (255, 0, 0), cv2.FILLED)
                if length<20:
                    cv2.circle(img,(zpos1, zpos2), 15, (0, 0, 255), cv2.FILLED)
            
            volRange = volume.GetVolumeRange()
            minVol = volRange[0]
            maxVol = volRange[1]
           
            # from numpy we find our length,by converting hand range in terms of volume range ie b/w -63.5 to 0
            vol = np.interp(length, [20, 250], [minVol, maxVol])
            volBar = np.interp(length, [20 ,250] , [400 ,150])
            volPer = np.interp(length, [20 ,250] , [0 ,100])
            volume.SetMasterVolumeLevel(vol, None)
            
            #volume bar for volume level
            cv2.rectangle(img , (50 ,150) , (85 , 400) ,(255, 0, 0) ,4)
            cv2.rectangle(img , (50 , int(volBar)) , (85 ,400) ,(255, 0, 0) ,cv2.FILLED)
            cv2.putText(img , str(int(volPer)) , (30, 450) ,cv2.FONT_HERSHEY_PLAIN ,4 , (0,0,255) , 3)
    
    cv2.imshow("Camera on", img)
    cv2.waitKey(1)
 

Error: Canceled future for execute_request message before replies were done